In [1]:
import pandas as pd

df = pd.read_excel("clinical_trials_tracker_20250403.xlsx")
print(df["status_history"].head(), df.shape)

0    [{'status_date': '2023-08-31T00:00:00.000002',...
1    [{'status_date': '2022-06-16T00:00:00.000004',...
2    [{'status_date': '2021-01-06T00:00:00.000002',...
3    [{'status_date': '2023-07-21T00:00:00.000002',...
4    [{'status_date': '2019-08-26T00:00:00.000002',...
Name: status_history, dtype: object (24415, 40)


In [4]:
dfsub = df.loc[:, ["nci_id", "status_history"]]


def decode(el: str):
    return eval(el)


def get_prop(el: dict, prop: str):
    return el[prop]


dfsub["status_history"] = dfsub["status_history"].apply(decode)
dfsubexp = dfsub.explode("status_history")
dfsubexp["status"] = dfsubexp["status_history"].apply(get_prop, args=("status",))
dfsubexp["status_date"] = dfsubexp["status_history"].apply(
    get_prop, args=("status_date",)
)
dfsubexp = dfsubexp.drop("status_history", axis=1)
expected_len = dfsub["status_history"].transform(len)
assert expected_len.sum() == len(dfsubexp)
dfsubexp.sort_values(["nci_id", "status_date"]).reset_index(drop=True).to_csv(
    "all_trials_status_hist_20250403.csv", index=False, encoding="utf_8_sig"
)
dfsubexp.sort_values(["nci_id", "status_date"]).reset_index(drop=True)

,nci_id,status,status_date
0,NCI-2009-00170,IN_REVIEW,2005-11-21T00:00:00
1,NCI-2009-00170,APPROVED,2006-06-19T00:00:00.000001
2,NCI-2009-00170,ACTIVE,2006-10-23T00:00:00.000002
3,NCI-2009-00170,CLOSED_TO_ACCRUAL,2015-11-10T00:00:00.000003
4,NCI-2009-00170,CLOSED_TO_ACCRUAL_AND_INTERVENTION,2023-07-24T00:00:00.000004
...,...,...,...
90416,NCI-2025-02348,APPROVED,2024-05-30T00:00:00
90417,NCI-2025-02348,ACTIVE,2024-07-16T00:00:00.000001
90418,NCI-2025-02348,TEMPORARILY_CLOSED_TO_ACCRUAL,2024-07-17T00:00:00.000002
90419,NCI-2025-02348,ACTIVE,2024-09-16T00:00:00.000003


In [5]:
finaldf = dfsubexp.sort_values(by="status_date").groupby(["nci_id", "status"]).nth(0)
finaldf = finaldf.reset_index(drop=True)
finaldf.sort_values(["nci_id", "status_date"]).to_csv(
    "all_trials_earliest_status_20250403.csv",
    index=False,
    encoding="utf_8_sig",
)